# Goodhart's Law: Experiment Runner

This notebook runs training experiments for the Goodhart's Law project.
Works on **Google Colab**, **VS Code Dev Containers**, and **Local Environments**.

## 1. Environment Setup

In [ ]:
import os
import sys
import subprocess

# --- Configuration ---
BRANCH = 'goodhartian'  # Branch to use (change to 'master' for stable releases)
USE_DRIVE = True        # Mount Google Drive for persistent storage
DRIVE_FOLDER = 'goodharts_law'  # Folder name in your Google Drive

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("[Colab] Detected Google Colab Environment")
    
    # Mount Google Drive if requested
    if USE_DRIVE:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_PATH = f'/content/drive/MyDrive/{DRIVE_FOLDER}'
        os.makedirs(DRIVE_PATH, exist_ok=True)
        print(f"[Colab] Google Drive mounted. Output folder: {DRIVE_PATH}")
    else:
        DRIVE_PATH = None
    
    # Clone the repository if not already present
    if not os.path.exists('goodharts_law'):
        print(f"[Colab] Cloning repository (branch: {BRANCH})...")
        !git clone --branch {BRANCH} https://github.com/FIM43-Redeye/goodharts_law.git
    else:
        print(f"[Colab] Repository already cloned. Checking out {BRANCH} and pulling...")
        %cd goodharts_law
        !git fetch origin
        !git checkout {BRANCH}
        !git pull origin {BRANCH}
        %cd ..
    
    # Install in editable mode for proper imports
    print("[Colab] Installing package in editable mode...")
    %cd goodharts_law
    %pip install -e . -q
    
    # Create symlinks to Google Drive for persistent storage
    if DRIVE_PATH:
        # Symlink models/ and logs/ to Drive
        for folder in ['models', 'logs']:
            local_path = folder
            drive_path = f'{DRIVE_PATH}/{folder}'
            os.makedirs(drive_path, exist_ok=True)
            if os.path.islink(local_path):
                os.unlink(local_path)
            elif os.path.isdir(local_path):
                # Move existing files to Drive before linking
                import shutil
                for item in os.listdir(local_path):
                    src = os.path.join(local_path, item)
                    dst = os.path.join(drive_path, item)
                    if not os.path.exists(dst):
                        shutil.move(src, dst)
                os.rmdir(local_path)
            os.symlink(drive_path, local_path)
            print(f"[Colab] {folder}/ -> {drive_path}")
    
    print("[Colab] Setup Complete")
else:
    print("[Local] Detected Local / Dev Container Environment")
    DRIVE_PATH = None
    # Assume already installed via pip install -e . or running from project root

## 2. Verify Environment
Check device and package installation.

In [ ]:
import torch
from goodharts.utils.device import get_device

device = get_device()
print(f"Running on device: {device}")
if torch.cuda.is_available():
    print(f"GPU Model: {torch.cuda.get_device_name(0)}")

## 3. Run Experiment
Configure parameters and run training via CLI.

In [ ]:
# --- Configuration ---
MODE = 'ground_truth'       # Options: ground_truth, ground_truth_handhold, proxy_ill_adjusted, etc.
TIMESTEPS = 50_000          # Total training steps
N_ENVS = 128                # Parallel environments (higher = faster, more RAM)
USE_AMP = True              # Automatic Mixed Precision (faster on GPU)

# Build command
amp_flag = '--use-amp' if USE_AMP else '--no-amp'

print(f"Starting Training: {MODE} ({TIMESTEPS:,} steps)")
!python -m goodharts.training.train_ppo --mode {MODE} --timesteps {TIMESTEPS} --n-envs {N_ENVS} {amp_flag}

## 4. Analysis
Visualize the training results using the built-in log viewer.

In [ ]:
# View training logs for the mode we just trained
!python -m goodharts.training.train_dashboard --latest --mode {MODE}